In [ ]:
""" MODELO PRE ENTRENADO SwinViT Small
Dataset: Patch Camelyon, de tensorflow preparado para HuggingFace
Fuente del modelo: HuggingFace
Modelo: microsoft/swin-small-patch4-window7-224
"""

' MODELO PRE ENTRENADO SwinViT Smal\nDataset: Patch Camelyon, de tensorflow preparado para HuggingFace\nFuente del modelo: HuggingFace\nModelo: microsoft/swin-small-patch4-window7-224\n'

In [ ]:
# Conectar con Google Drivee()
from google.colab import drive
drive.mount('/content/drive')
base_folder = "/content/drive/MyDrive/00 VIU/10 TFM"


Mounted at /content/drive


In [ ]:
# librerías
!pip install -q datasets evaluate keras_cv
!pip install --upgrade transformers
!pip install -q tensorflow
!pip install -q tf-keras


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 650.7/650.7 kB 27.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 950.8/950.8 kB 41.7 MB/s eta 0:00:00


In [ ]:
import numpy as np
import transformers
import evaluate
from transformers import AutoModelForImageClassification, AutoImageProcessor,TrainingArguments, Trainer
from datasets import Dataset as HuggingFaceDataset, Features
from PIL import Image as PILImage
import torch

In [ ]:
# importar credenciales de Hugging face
from google.colab import userdata
import os
from huggingface_hub import login

# Accede al token almacenado en los secretos de Colab
hf_token = userdata.get('HF_TOKEN')

## Cargar el dataset patch camelyon de hugging face

In [ ]:
train_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/train 12k")

val_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/val 12k")

## Preprocesar los datos

In [ ]:
modelo = "microsoft/swin-small-patch4-window7-224"
batch_size = 32

# instanciar el image processing
image_processor = AutoImageProcessor.from_pretrained(modelo,do_rescale=True,use_fast=True)

preprocessor_config.json:   0%|          | 0.00/255 [00:00<?, ?B/s]

In [ ]:
def preprocess_hf_dataset(examples):
    images_pil = [img for img in examples["image"]]
    inputs = image_processor(images=images_pil, return_tensors="pt")

    # inputs["pixel_values"] es un solo tensor PyTorch de forma (batch_size_interno, C, H, W)
    # Debemos dividirlo en una lista de tensores individuales (C, H, W), uno por cada imagen
    # para que el Dataset de Hugging Face lo almacene correctamente por ejemplo.
    # .unbind(0) divide el tensor a lo largo de la dimensión 0 (batch_size)
    pixel_values_list = list(inputs["pixel_values"].unbind(0))

    # `examples["label"]` es una lista de Python `int`s cuando `batched=True`
    labels_list = examples["label"]

    return {"pixel_values": pixel_values_list, "labels": labels_list}

In [ ]:
train_hf_ds = train_hf_ds.map(preprocess_hf_dataset, batched=True, remove_columns=["image"],load_from_cache_file=False)
val_hf_ds = val_hf_ds.map(preprocess_hf_dataset, batched=True, remove_columns=["image"],load_from_cache_file=False)

Map:   0%|          | 0/12000 [00:00<?, ? examples/s]

Map:   0%|          | 0/2400 [00:00<?, ? examples/s]

In [ ]:
num_labels = 2
model = AutoModelForImageClassification.from_pretrained(
    modelo,
    num_labels=num_labels,
    ignore_mismatched_sizes = True
)

config.json:   0%|          | 0.00/71.8k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/199M [00:00<?, ?B/s]

Some weights of SwinForImageClassification were not initialized from the model checkpoint at microsoft/swin-small-patch4-window7-224 and are newly initialized because the shapes did not match:
- classifier.weight: found shape torch.Size([1000, 768]) in the checkpoint and torch.Size([2, 768]) in the model instantiated
- classifier.bias: found shape torch.Size([1000]) in the checkpoint and torch.Size([2]) in the model instantiated
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
print(model)

SwinForImageClassification(
  (swin): SwinModel(
    (embeddings): SwinEmbeddings(
      (patch_embeddings): SwinPatchEmbeddings(
        (projection): Conv2d(3, 96, kernel_size=(4, 4), stride=(4, 4))
      )
      (norm): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.0, inplace=False)
    )
    (encoder): SwinEncoder(
      (layers): ModuleList(
        (0): SwinStage(
          (blocks): ModuleList(
            (0): SwinLayer(
              (layernorm_before): LayerNorm((96,), eps=1e-05, elementwise_affine=True)
              (attention): SwinAttention(
                (self): SwinSelfAttention(
                  (query): Linear(in_features=96, out_features=96, bias=True)
                  (key): Linear(in_features=96, out_features=96, bias=True)
                  (value): Linear(in_features=96, out_features=96, bias=True)
                  (dropout): Dropout(p=0.0, inplace=False)
                )
                (output): SwinSelfOutput(
        

In [ ]:
print(model.config)

SwinConfig {
  "architectures": [
    "SwinForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    18,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 96,
  "encoder_stride": 32,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "swin",
  "num_channels": 3,
  "num_heads": [
    3,
    6,
    12,
    24
  ],
  "num_layers": 4,
  "out_features": [
    "stage4"
  ],
  "out_indices": [
    4
  ],
  "patch_size": 4,
  "path_norm": true,
  "qkv_bias": true,
  "stage_names": [
    "stem",
    "stage1",
    "stage2",
    "stage3",
    "stage4"
  ],
  "torch_dtype": "float32",
  "transformers_version": "4.52.4",
  "use_absolute_embeddings": false,
  "window_size": 7
}



In [ ]:
print("Cantidad de parámetros del modelo: ",(sum(p.numel() for p in model.parameters())//1e5)/10, "M")  # total parámetros


Cantidad de parámetros del modelo:  48.8 M


In [ ]:
metric = evaluate.load("accuracy")
def compute_metrics(p):
    predictions = np.argmax(p.predictions, axis=1)
    references = p.label_ids
    metrics_result = metric.compute(predictions=predictions, references=references)
    print(f"Metric computation result: {metrics_result}")
    return {"eval_accuracy": metrics_result["accuracy"]}

model.safetensors:   0%|          | 0.00/199M [00:00<?, ?B/s]

In [ ]:
# establecer los parámetros del entrenamiento
args = TrainingArguments(
    remove_unused_columns=False,
    eval_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=3,
    load_best_model_at_end=True,
    metric_for_best_model="eval_accuracy",
    report_to="none",
    output_dir=base_folder+"/Modelos entrenados/Swin v1 Small"
)

### Collator

In [ ]:
import torch

def custom_image_collator(batch):
    processed_pixel_values_for_batch = []
    for example in batch:
        current_pixel_values = example["pixel_values"] # Esto es una <class 'list'>
        if isinstance(current_pixel_values, list):
            temp_tensor = None
            if len(current_pixel_values) > 0:
                if torch.is_tensor(current_pixel_values[0]):
                    if len(current_pixel_values) == 3 and all(torch.is_tensor(x) for x in current_pixel_values):
                        temp_tensor = torch.stack(current_pixel_values) # stackea canales: (C, H, W)
                    elif len(current_pixel_values) == 1 and torch.is_tensor(current_pixel_values[0]):
                        temp_tensor = current_pixel_values[0]
                    else:
                        raise ValueError(f"Unexpected list of Tensors structure for pixel_values: {current_pixel_values[0].shape if len(current_pixel_values) > 0 else 'empty list'}")
                elif isinstance(current_pixel_values[0], list):
                    temp_tensor = torch.tensor(current_pixel_values, dtype=torch.float32) # Convertir la lista anidada a tensor
                else:
                    raise ValueError(f"Unexpected element type in pixel_values list: {type(current_pixel_values[0])}. Expected Tensor or list.")
            else:
                raise ValueError("pixel_values list is empty.")
        elif torch.is_tensor(current_pixel_values):
            temp_tensor = current_pixel_values
        else:
            raise ValueError(f"Unexpected main type for pixel_values: {type(current_pixel_values)}. Expected list or Tensor.")

        processed_pixel_values_for_batch.append(temp_tensor)

    stacked_pixel_values = torch.stack(processed_pixel_values_for_batch)


    labels_clean = []
    for example in batch:
        if isinstance(example["labels"], torch.Tensor):
            labels_clean.append(example["labels"].item() if example["labels"].numel() == 1 else example["labels"].tolist()[0])
        else:
            labels_clean.append(example["labels"])
    stacked_labels = torch.tensor(labels_clean, dtype=torch.long)

    return {"pixel_values": stacked_pixel_values, "labels": stacked_labels}

In [ ]:
trainer = Trainer(
    model=model,
    args=args,
    train_dataset=train_hf_ds,
    eval_dataset=val_hf_ds,
    compute_metrics=compute_metrics,
    data_collator=custom_image_collator,
    tokenizer=image_processor
)

/tmp/ipython-input-17-546105683.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(


In [ ]:
%%time
print("\n--- Iniciando entrenamiento ---")
train_results = trainer.train()


--- Iniciando entrenamiento ---


Epoch,Training Loss,Validation Loss,Accuracy
1,No log,0.170051,0.930833
2,No log,0.141129,0.951250
3,No log,0.142989,0.956667


Metric computation result: {'accuracy': 0.9308333333333333}
Metric computation result: {'accuracy': 0.95125}
Metric computation result: {'accuracy': 0.9566666666666667}
CPU times: user 1h 13min 35s, sys: 3min 29s, total: 1h 17min 5s
Wall time: 1h 20min 12s


In [ ]:
trainer.save_model(base_folder+"/Modelos entrenados/Swin Small v1")

In [ ]:
# EVALUAR EL MODELO

# Cargar el dataset de test 32k
test_hf_ds = HuggingFaceDataset.load_from_disk(base_folder+"/Datasets/pcamelyon_hf/test")

# Preprocesar el dataset de test
test_hf_ds = test_hf_ds.map(preprocess_hf_dataset, batched=True, remove_columns=["image"], load_from_cache_file=False)

# Evaluar el modelo con el dataset de test


/usr/local/lib/python3.11/dist-packages/datasets/table.py:1421: FutureWarning: promote has been superseded by promote_options='default'.
  table = cls._concat_blocks(blocks, axis=0)


Map:   0%|          | 0/32768 [00:00<?, ? examples/s]

In [ ]:
%%time
test_results = trainer.evaluate(test_hf_ds)

print("\n--- Resultados de la evaluación en el dataset de test ---")
print(test_results)


Metric computation result: {'accuracy': 0.88232421875}

--- Resultados de la evaluación en el dataset de test ---
{'eval_accuracy': 0.88232421875, 'eval_loss': 0.36517882347106934, 'eval_runtime': 3283.206, 'eval_samples_per_second': 9.98, 'eval_steps_per_second': 0.312, 'epoch': 3.0}
CPU times: user 54min, sys: 20.3 s, total: 54min 20s
Wall time: 54min 43s


In [ ]:
print(model.config)

SwinConfig {
  "architectures": [
    "SwinForImageClassification"
  ],
  "attention_probs_dropout_prob": 0.0,
  "depths": [
    2,
    2,
    18,
    2
  ],
  "drop_path_rate": 0.1,
  "embed_dim": 96,
  "encoder_stride": 32,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.0,
  "hidden_size": 768,
  "image_size": 224,
  "initializer_range": 0.02,
  "layer_norm_eps": 1e-05,
  "mlp_ratio": 4.0,
  "model_type": "swin",
  "num_channels": 3,
  "num_heads": [
    3,
    6,
    12,
    24
  ],
  "num_layers": 4,
  "out_features": [
    "stage4"
  ],
  "out_indices": [
    4
  ],
  "patch_size": 4,
  "path_norm": true,
  "problem_type": "single_label_classification",
  "qkv_bias": true,
  "stage_names": [
    "stem",
    "stage1",
    "stage2",
    "stage3",
    "stage4"
  ],
  "torch_dtype": "float32",
  "transformers_version": "4.52.4",
  "use_absolute_embeddings": false,
  "window_size": 7
}



In [ ]:
total_params = sum(p.numel() for p in model.parameters())
print(f"Total parameters: {total_params}")

Total parameters: 48838796
